# Guía 4: Aplicaciones Prácticas de Transformers

## Descripción

En esta guía vas a trabajar con aplicaciones reales de modelos Transformer usando la biblioteca Transformers de Hugging Face. A diferencia de las guías anteriores donde exploramos cómo funcionan internamente los modelos, acá vamos a usarlos como herramientas listas para resolver problemas concretos de procesamiento del lenguaje natural.

Al finalizar esta guía vas a poder:
- Usar pipelines de Hugging Face para tareas comunes de NLP
- Aplicar análisis de sentimiento en español
- Implementar clasificación zero-shot sin entrenar modelos
- Generar resúmenes automáticos de textos
- Realizar traducción automática
- Generar texto creativo con modelos GPT
- Evaluar cuándo usar cada tipo de modelo según la tarea

---

## Configuración del Entorno

### Requisitos

Para ejecutar este notebook necesitás:

1. **Google Colab con GPU activada:**
   - Andá a `Entorno de ejecución > Cambiar tipo de entorno de ejecución`
   - Seleccioná `Acelerador de hardware > GPU`
   - Elegí `Tipo de GPU > T4`

2. **Conexión a Internet** para descargar modelos preentrenados

Nota: Algunos ejemplos funcionan en CPU, pero la GPU acelera significativamente la inferencia.

## Instalación de Dependencias

Instalamos la biblioteca Transformers de Hugging Face, que proporciona acceso a miles de modelos preentrenados.

In [1]:
# Instalación silenciosa de la biblioteca
!pip install -q transformers

### ¿Qué es Transformers?

**Transformers** es una biblioteca de código abierto desarrollada por Hugging Face que proporciona:

- **Miles de modelos preentrenados**: BERT, GPT, T5, RoBERTa, y muchos más
- **Pipelines simplificados**: Interfaces de alto nivel para tareas comunes
- **Soporte multilingüe**: Modelos entrenados en múltiples idiomas, incluyendo español
- **Compatibilidad**: Funciona con PyTorch, TensorFlow y JAX

Ventajas:
- No necesitás entrenar modelos desde cero
- Podés aplicar modelos de última generación con pocas líneas de código
- La comunidad contribuye constantemente con nuevos modelos

## Marco Teórico

### Pipelines de Hugging Face

Un **pipeline** encapsula todo el flujo de trabajo para una tarea de NLP:

1. **Preprocesamiento**: Tokenización del texto de entrada
2. **Inferencia**: Ejecución del modelo
3. **Postprocesamiento**: Conversión de la salida a formato legible

Esto te permite usar modelos complejos sin preocuparte por los detalles de implementación.

### Tareas Comunes de NLP

**1. Análisis de Sentimiento**
- Determina si un texto expresa sentimiento positivo, negativo o neutral
- Aplicaciones: Análisis de reviews, monitoreo de redes sociales, feedback de clientes

**2. Clasificación Zero-Shot**
- Clasifica textos en categorías sin entrenamiento específico
- El modelo infiere la categoría basándose en su comprensión del lenguaje
- Aplicaciones: Categorización de documentos, filtrado de contenido

**3. Sumarización**
- Genera resúmenes concisos de textos largos
- Tipos: Extractiva (selecciona oraciones) vs Abstractiva (genera nuevas oraciones)
- Aplicaciones: Resúmenes de noticias, síntesis de documentos, abstracts automáticos

**4. Traducción Automática**
- Traduce texto entre idiomas
- Modelos modernos: Basados en Transformer (superan a sistemas estadísticos clásicos)
- Aplicaciones: Localización de software, traducción de documentos

**5. Generación de Texto**
- Produce texto nuevo a partir de un prompt
- Modelos: GPT-2, GPT-3, y variantes entrenadas en idiomas específicos
- Aplicaciones: Asistentes de escritura, generación de contenido, chatbots

### Modelos en Español

Para trabajar efectivamente con texto en español, usamos modelos específicamente entrenados o ajustados con datos en español:

- **BETO**: BERT entrenado en español (clasificación, NER)
- **MarianMT**: Familia de modelos de traducción (Helsinki-NLP)
- **mT5**: Modelo multilingüe para sumarización y tareas seq2seq
- **GPT-2 Spanish**: Versiones de GPT-2 entrenadas en español (PlanTL-GOB-ES)

## Parte 1: Análisis de Sentimiento en Español

Vamos a usar un modelo entrenado específicamente para identificar sentimientos en tweets en español.

In [2]:
from transformers import pipeline

# Cargamos el pipeline de análisis de sentimiento
# model: Especifica qué modelo usar (finiteautomata/beto-sentiment-analysis está entrenado con tweets argentinos)
sentiment = pipeline(
    "sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Pipeline de sentimiento cargado exitosamente")

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Pipeline de sentimiento cargado exitosamente


### Modelo BETO Sentiment

**finiteautomata/beto-sentiment-analysis** es un modelo basado en BETO (BERT en español) entrenado con tweets en español, particularmente de Argentina.

Características:
- Clasifica en tres categorías: POS (positivo), NEG (negativo), NEU (neutral)
- Entrenado con lenguaje coloquial y expresiones regionales
- Maneja bien sarcasmo y expresiones informales
- Devuelve una etiqueta y un score de confianza (0-1)

In [3]:
# Frases de prueba con diferentes sentimientos
frases = [
    "Este lugar está buenísimo, la atención de diez",
    "Una experiencia horrible, me quiero ir",
    "Zafa, pero esperaba más",
    "Recomiendo totalmente este producto",
    "Nunca más compro acá",
    "El asado estaba en su punto, espectacular",
    "Meh, ni fu ni fa"
]

print("Análisis de Sentimiento:")
print("=" * 80)

for frase in frases:
    resultado = sentiment(frase)[0]  # El pipeline devuelve una lista, tomamos el primer elemento
    etiqueta = resultado['label']     # Etiqueta: POS, NEG, NEU
    confianza = resultado['score']    # Score de confianza (0-1)

    print(f"\nTexto: '{frase}'")
    print(f"Sentimiento: {etiqueta} (confianza: {confianza:.3f})")

Análisis de Sentimiento:

Texto: 'Este lugar está buenísimo, la atención de diez'
Sentimiento: POS (confianza: 0.999)

Texto: 'Una experiencia horrible, me quiero ir'
Sentimiento: NEG (confianza: 0.999)

Texto: 'Zafa, pero esperaba más'
Sentimiento: NEG (confianza: 0.926)

Texto: 'Recomiendo totalmente este producto'
Sentimiento: POS (confianza: 0.999)

Texto: 'Nunca más compro acá'
Sentimiento: NEG (confianza: 0.999)

Texto: 'El asado estaba en su punto, espectacular'
Sentimiento: POS (confianza: 0.997)

Texto: 'Meh, ni fu ni fa'
Sentimiento: NEG (confianza: 0.962)


### Análisis de Resultados

Observá cómo el modelo:
- Identifica expresiones positivas como "buenísimo", "de diez", "espectacular"
- Reconoce negatividad en "horrible", "nunca más"
- Detecta neutralidad en "zafa", "ni fu ni fa"

El **score de confianza** indica qué tan seguro está el modelo:
- > 0.9: Alta confianza (sentimiento claro)
- 0.7-0.9: Confianza moderada
- < 0.7: Baja confianza (sentimiento ambiguo)

In [4]:
# Ejercicio: Probá con tus propias frases
mis_frases = [
    # Agregá tus propias frases aquí para analizar
    "Tu frase 1",
    "Tu frase 2",
]

for frase in mis_frases:
    resultado = sentiment(frase)[0]
    print(f"{frase} → {resultado['label']} ({resultado['score']:.3f})")

Tu frase 1 → POS (0.998)
Tu frase 2 → POS (0.998)


## Parte 2: Clasificación Zero-Shot

La clasificación zero-shot te permite categorizar textos sin necesidad de entrenar un modelo con ejemplos de cada categoría. El modelo usa su comprensión del lenguaje para inferir la categoría más apropiada.

In [5]:
# Cargamos el pipeline de clasificación zero-shot
# Este modelo fue entrenado con el dataset XNLI (cross-lingual natural language inference)
classifier = pipeline(
    "zero-shot-classification",
    model="Recognai/bert-base-spanish-wwm-cased-xnli"
)

print("Pipeline de clasificación zero-shot cargado")

config.json:   0%|          | 0.00/834 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Pipeline de clasificación zero-shot cargado


### ¿Cómo Funciona Zero-Shot?

El modelo fue entrenado en una tarea de **inferencia de lenguaje natural (NLI)**:
- Dadas dos oraciones (premisa e hipótesis), determina si la hipótesis se deduce de la premisa

Para clasificación:
1. Tu texto es la "premisa"
2. Cada etiqueta candidata se convierte en una "hipótesis"
3. El modelo calcula qué hipótesis es más probable dada la premisa

Ejemplo:
- Texto: "Messi marcó tres goles"
- Etiqueta "deportes" → Hipótesis: "Este texto trata sobre deportes"
- Etiqueta "política" → Hipótesis: "Este texto trata sobre política"
- El modelo determina que la primera hipótesis es más probable

In [6]:
# Texto a clasificar
texto = "Lionel Messi firmó contrato con el Inter Miami y debutará esta semana en la MLS."

# Categorías candidatas (podés usar las categorías que quieras)
etiquetas = ["deportes", "economía", "política", "espectáculos", "tecnología"]

# Clasificamos
# candidate_labels: Lista de posibles categorías
resultado = classifier(texto, candidate_labels=etiquetas)

print(f"Texto: '{texto}'\n")
print("Clasificación:")
print("=" * 60)

# El resultado incluye las etiquetas ordenadas por score
for etiqueta, score in zip(resultado['labels'], resultado['scores']):
    print(f"{etiqueta:15} | {score:.4f} ({score*100:.2f}%)")

Texto: 'Lionel Messi firmó contrato con el Inter Miami y debutará esta semana en la MLS.'

Clasificación:
deportes        | 0.2915 (29.15%)
espectáculos    | 0.2856 (28.56%)
tecnología      | 0.2136 (21.36%)
política        | 0.1185 (11.85%)
economía        | 0.0908 (9.08%)


In [7]:
# Probemos con más ejemplos
textos_prueba = [
    "El Congreso aprobó la nueva ley de presupuesto después de intensas negociaciones entre los bloques.",
    "Apple presentó su nuevo iPhone con pantalla plegable y cámara de 200 megapíxeles.",
    "La obra de teatro recibió tres premios en el festival internacional de artes escénicas.",
    "El Banco Central decidió mantener la tasa de interés en 75% para controlar la inflación."
]

for texto in textos_prueba:
    resultado = classifier(texto, candidate_labels=etiquetas)
    categoria_principal = resultado['labels'][0]
    confianza = resultado['scores'][0]

    print(f"\nTexto: '{texto[:70]}...'")
    print(f"Categoría: {categoria_principal} (confianza: {confianza:.3f})")


Texto: 'El Congreso aprobó la nueva ley de presupuesto después de intensas neg...'
Categoría: política (confianza: 0.456)

Texto: 'Apple presentó su nuevo iPhone con pantalla plegable y cámara de 200 m...'
Categoría: tecnología (confianza: 0.617)

Texto: 'La obra de teatro recibió tres premios en el festival internacional de...'
Categoría: espectáculos (confianza: 0.389)

Texto: 'El Banco Central decidió mantener la tasa de interés en 75% para contr...'
Categoría: política (confianza: 0.296)


### Ventajas y Limitaciones de Zero-Shot

**Ventajas:**
- No requiere datos de entrenamiento etiquetados
- Flexible: podés cambiar las categorías sin reentrenar
- Útil para prototipado rápido

**Limitaciones:**
- Menos preciso que modelos entrenados específicamente para tu tarea
- Sensible a cómo formulás las etiquetas
- Puede ser más lento que clasificadores tradicionales

In [8]:
# Ejercicio: Clasificá tus propios textos
mi_texto = "Tu texto aquí"
mis_categorias = ["categoría1", "categoría2", "categoría3"]

resultado = classifier(mi_texto, candidate_labels=mis_categorias)
print(f"Texto: {mi_texto}")
print(f"Categoría predicha: {resultado['labels'][0]} ({resultado['scores'][0]:.3f})")

Texto: Tu texto aquí
Categoría predicha: categoría2 (0.456)


## Parte 3: Sumarización Automática

Vamos a generar resúmenes concisos de textos largos usando un modelo multilingüe de sumarización.

In [9]:
# Cargamos el pipeline de sumarización
# mT5 es un modelo multilingüe entrenado en más de 100 idiomas
summarizer = pipeline(
    "summarization",
    model="csebuetnlp/mT5_multilingual_XLSum",
    tokenizer="csebuetnlp/mT5_multilingual_XLSum"
)

print("Pipeline de sumarización cargado")

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


Pipeline de sumarización cargado


### mT5 XLSum

**mT5** (multilingual T5) es un modelo encoder-decoder entrenado para tareas de transformación de texto.

El modelo **XLSum** fue específicamente ajustado para generar resúmenes de una oración (extreme summarization) en múltiples idiomas.

Características:
- Sumarización **abstractiva**: Genera nuevas oraciones, no solo extrae del original
- Multilingüe: Funciona con español, inglés, y muchos otros idiomas
- Resúmenes concisos: Típicamente genera 1-3 oraciones

In [10]:
# Texto de ejemplo (noticia)
texto_largo = """
El Ministerio de Salud confirmó hoy que se logró una reducción sostenida de casos de dengue
en las últimas semanas. Según los datos provistos, hubo una disminución del 40% en comparación
con el pico de marzo. Las campañas de prevención, sumadas a la llegada del frío, contribuyeron
a esta baja. Sin embargo, las autoridades sanitarias piden a la población mantener las precauciones,
especialmente eliminando recipientes con agua estancada donde el mosquito puede reproducirse.
El director de Epidemiología destacó que, aunque los números son alentadores, es fundamental no
bajar la guardia. Se espera que con el invierno los casos continúen descendiendo, pero la
vigilancia epidemiológica seguirá activa.
"""

# Generamos el resumen
# max_length: Longitud máxima del resumen en tokens
# min_length: Longitud mínima del resumen en tokens
# do_sample: False para generación determinística
resumen = summarizer(
    texto_largo,
    max_length=50,
    min_length=20,
    do_sample=False
)

print("Texto original:")
print(texto_largo.strip())
print("\n" + "="*80 + "\n")
print("Resumen generado:")
print(resumen[0]['summary_text'])

Texto original:
El Ministerio de Salud confirmó hoy que se logró una reducción sostenida de casos de dengue 
en las últimas semanas. Según los datos provistos, hubo una disminución del 40% en comparación 
con el pico de marzo. Las campañas de prevención, sumadas a la llegada del frío, contribuyeron 
a esta baja. Sin embargo, las autoridades sanitarias piden a la población mantener las precauciones, 
especialmente eliminando recipientes con agua estancada donde el mosquito puede reproducirse. 
El director de Epidemiología destacó que, aunque los números son alentadores, es fundamental no 
bajar la guardia. Se espera que con el invierno los casos continúen descendiendo, pero la 
vigilancia epidemiológica seguirá activa.


Resumen generado:
El Ministerio de Salud confirmó este jueves que se logró una reducción sostenida de casos de dengue en las últimas semanas.


### Parámetros de Sumarización

**max_length y min_length:**
- Controlan la longitud del resumen en tokens
- Ajustalos según cuán conciso querés el resumen
- Si el texto original es corto, max_length debe ser proporcionalmente menor

**do_sample:**
- `False`: Generación determinística (siempre el mismo resumen)
- `True`: Introduce variación (diferentes resúmenes en cada ejecución)

**Otros parámetros útiles:**
- `length_penalty`: Penaliza resúmenes muy largos o cortos (default: 1.0)
- `num_beams`: Cantidad de secuencias a explorar simultáneamente (beam search)
- `early_stopping`: Detiene cuando encuentra un buen resumen (con beam search)

In [11]:
# Probemos con otro texto
texto_economia = """
La inflación en Argentina mostró una leve desaceleración en el último mes, según el informe
del INDEC publicado esta mañana. El índice de precios al consumidor registró un aumento del
7.2%, por debajo del 8.4% del mes anterior. Los analistas atribuyen esta baja a la política
monetaria restrictiva implementada por el Banco Central y a la estabilización del tipo de
cambio. Sin embargo, advierten que la tendencia aún no se revirtió de manera definitiva, y
podrían esperarse nuevos aumentos en los próximos meses si no se sostienen las medidas actuales.
Los sectores más afectados continúan siendo alimentos y bebidas, con incrementos superiores al
promedio general.
"""

resumen_economia = summarizer(
    texto_economia,
    max_length=60,
    min_length=25,
    do_sample=False
)

print("Texto original:")
print(texto_economia.strip())
print("\n" + "="*80 + "\n")
print("Resumen:")
print(resumen_economia[0]['summary_text'])

Texto original:
La inflación en Argentina mostró una leve desaceleración en el último mes, según el informe 
del INDEC publicado esta mañana. El índice de precios al consumidor registró un aumento del 
7.2%, por debajo del 8.4% del mes anterior. Los analistas atribuyen esta baja a la política 
monetaria restrictiva implementada por el Banco Central y a la estabilización del tipo de 
cambio. Sin embargo, advierten que la tendencia aún no se revirtió de manera definitiva, y 
podrían esperarse nuevos aumentos en los próximos meses si no se sostienen las medidas actuales. 
Los sectores más afectados continúan siendo alimentos y bebidas, con incrementos superiores al 
promedio general.


Resumen:
La inflación en Argentina mostró una leve desaceleración en el último mes, según un informe del Instituto Nacional de Economía (INDEC).


In [12]:
# Ejercicio: Resumí tu propio texto
mi_texto_largo = """
Tu texto largo aquí. Puede ser una noticia, un artículo,
o cualquier texto que quieras resumir.
"""

mi_resumen = summarizer(
    mi_texto_largo,
    max_length=50,
    min_length=20,
    do_sample=False
)

print("Resumen:")
print(mi_resumen[0]['summary_text'])

Your max_length is set to 50, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


Resumen:
¿Cuántas veces has escuchado hablar de noticias relacionadas con el coronavirus?


## Parte 4: Traducción Automática

Vamos a traducir texto del español al inglés usando modelos de la familia MarianMT.

In [13]:
# Cargamos el pipeline de traducción español → inglés
# Helsinki-NLP/opus-mt-es-en: Modelo entrenado en el corpus OPUS
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-es-en"
)

print("Pipeline de traducción cargado")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


Pipeline de traducción cargado


### MarianMT

**MarianMT** es una familia de modelos de traducción neural desarrollados por Microsoft y la Universidad de Edimburgo.

Características:
- Modelos específicos para cada par de idiomas (es→en, en→es, etc.)
- Entrenados en el corpus OPUS (millones de pares de oraciones paralelas)
- Relativamente rápidos y livianos
- Arquitectura encoder-decoder basada en Transformer

In [14]:
# Textos para traducir
textos_espanol = [
    "La inteligencia artificial está cambiando el mundo.",
    "El mate es la bebida tradicional de Argentina y Uruguay.",
    "Los modelos de lenguaje pueden generar texto coherente y contextualmente apropiado.",
    "Me encanta programar en Python porque es simple y poderoso."
]

print("Traducción Español → Inglés")
print("=" * 80)

for texto in textos_espanol:
    traduccion = translator(texto)[0]['translation_text']
    print(f"\nES: {texto}")
    print(f"EN: {traduccion}")

Traducción Español → Inglés

ES: La inteligencia artificial está cambiando el mundo.
EN: Artificial intelligence is changing the world.

ES: El mate es la bebida tradicional de Argentina y Uruguay.
EN: Matte is the traditional drink of Argentina and Uruguay.

ES: Los modelos de lenguaje pueden generar texto coherente y contextualmente apropiado.
EN: Language models can generate coherent and contextually appropriate text.

ES: Me encanta programar en Python porque es simple y poderoso.
EN: I love programming in Python because it's simple and powerful.


### Calidad de la Traducción

Los modelos de traducción neural modernos son muy superiores a los sistemas estadísticos anteriores, pero tienen limitaciones:

**Fortalezas:**
- Manejo de contexto: Entienden el significado de la oración completa
- Fluidez: Generan traducciones naturales y gramaticalmente correctas
- Idiomatismos: Pueden traducir expresiones coloquiales (aunque no siempre perfectamente)

**Limitaciones:**
- Expresiones muy regionales pueden traducirse literalmente
- Nombres propios pueden traducirse incorrectamente
- Pueden perder matices culturales
- Sensibles a la longitud: Oraciones muy largas pueden tener errores

In [15]:
# Probemos con expresiones coloquiales argentinas
expresiones = [
    "Me colgué y se me hizo tarde.",
    "La película estuvo de diez, te la súper recomiendo.",
    "Aguante el mate y las facturas."
]

print("Traduciendo expresiones coloquiales:")
print("=" * 80)

for expresion in expresiones:
    traduccion = translator(expresion)[0]['translation_text']
    print(f"\nES: {expresion}")
    print(f"EN: {traduccion}")
    print("Nota: Observá si mantiene el sentido original o traduce literalmente")

Traduciendo expresiones coloquiales:

ES: Me colgué y se me hizo tarde.
EN: I hung up and got late.
Nota: Observá si mantiene el sentido original o traduce literalmente

ES: La película estuvo de diez, te la súper recomiendo.
EN: The movie was ten, I highly recommend it.
Nota: Observá si mantiene el sentido original o traduce literalmente

ES: Aguante el mate y las facturas.
EN: Hang on to the math and bills.
Nota: Observá si mantiene el sentido original o traduce literalmente


In [16]:
# Ejercicio: Traducí tus propias frases
mis_textos = [
    "Tu texto 1 en español",
    "Tu texto 2 en español",
]

for texto in mis_textos:
    traduccion = translator(texto)[0]['translation_text']
    print(f"ES: {texto}")
    print(f"EN: {traduccion}\n")

ES: Tu texto 1 en español
EN: Your text 1 in Spanish

ES: Tu texto 2 en español
EN: Your text 2 in Spanish



## Parte 5: Generación de Texto con GPT

Finalmente, vamos a generar texto creativo usando un modelo GPT-2 entrenado en español.

In [17]:
# Cargamos el pipeline de generación de texto
# PlanTL-GOB-ES es el Plan de Tecnologías del Lenguaje del gobierno español
generator = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-3B-Instruct"
)

print("Pipeline de generación de texto cargado")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


Pipeline de generación de texto cargado


### GPT-2 en Español

**GPT-2** (Generative Pre-trained Transformer 2) es un modelo decoder-only diseñado para generar texto.

**gpt2-base-bne:**
- Entrenado en la Biblioteca Nacional de España (BNE)
- Corpus: Libros, periódicos, documentos históricos en español
- 124M parámetros (versión "base")
- Genera texto coherente continuando un prompt

A diferencia de modelos instruction-tuned como Phi-3, GPT-2 simplemente continúa el texto sin seguir instrucciones específicas.

In [18]:
# Prompts para generar texto
prompts = [
    "La inteligencia artificial",
    "En el año 2050",
    "El mate es",
    "Había una vez"
]

print("Generación de Texto")
print("=" * 80)

for prompt in prompts:
    # max_length: Longitud total (prompt + generado) en tokens
    # num_return_sequences: Cuántas variaciones generar
    # do_sample: True para introducir aleatoriedad
    # temperature: Controla creatividad (0.1-2.0)
    resultado = generator(
        prompt,
        max_length=50,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.8
    )

    print(f"\nPrompt: '{prompt}'")
    print(f"Generado: {resultado[0]['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generación de Texto


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: 'La inteligencia artificial'
Generado: La inteligencia artificial (IA) es una rama de la informática que se ocupa de la creación de sistemas capaces de realizar tareas complejas de forma automática, sin la intervención humana. A continuación, te presento algunos conceptos y aspectos importantes sobre la inteligencia artificial:

### 1. Tipos de Inteligencia Artificial

Existen varios tipos de inteligencia artificial, incluyendo:

*   **Inteligencia artificial débil (DA):** se enfoca en resolver problemas específicos y es capaz de aprender y mejorar con la experiencia.
*   **Inteligencia artificial fuerte (IAF):** se refiere a la creación de sistemas capaces de realizar cualquier tarea que un ser humano puede hacer, sin la intervención humana.
*   **Inteligencia artificial superinteligente:** se refiere a sistemas que son mucho más inteligentes que los humanos en una amplia gama de tareas.

### 2. Algoritmos y técnicas

La inteligencia artificial se basa en algoritmos y técnica

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: 'En el año 2050'
Generado: En el año 2050, la humanidad ha explorado y colonizado varios planetas y lunas en el sistema solar. La colonia más grande y próspera se encuentra en la luna de Marte, llamada Nova Terra. La colonia de Nova Terra ha sido fundada por una sociedad de ingenieros, científicos y empresarios que han desarrollado tecnologías avanzadas para vivir y trabajar en un entorno hostil.

La colonia de Nova Terra ha sido diseñada para ser autosuficiente, con sus propias fuentes de energía, agua y alimentos. La colonia cuenta con una capital, llamada Nova Haven, que es el centro de la vida política, económica y social de la colonia. La ciudad es un modelo de arquitectura y diseño, diseñada para ser sostenible y respetuosa con el entorno.

La colonia de Nova Terra también cuenta con una flota de naves espaciales que se utilizan para explorar y colonizar otros planetas y lunas en el sistema solar. La flota es liderada por la tripulación de la nave espacial "Aurora", que 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Prompt: 'El mate es'
Generado: El mate es una bebida tradicional argentina y uruguaya, que se toma en un gusano de bambú, similar a un calabazón. Se prepara de manera similar a la té, con hojas secas de mate (Ilex paraguariensis), y se bebe caliente. El mate es conocido por sus beneficios para la salud, como la reducción del colesterol, la prevención de enfermedades cardiovasculares y la mejora de la función inmunológica.

La cultura del mate se ha extendido por América Latina, y se considera una bebida nacional en Argentina y Uruguay. En estos países, se puede encontrar mate en casi cualquier lugar, desde cafeterías hasta mercados. La tradición del mate se ha preservado a través de la generación, y se considera un símbolo de la hospitalidad y la amistad.

En cuanto a la preparación del mate, hay varias formas de hacerlo. La más tradicional es el "mate tradicional", que consiste en colocar una cucharada de hojas secas de mate en un gusano de bambú, agregar agua caliente y revolver. Si

### Parámetros de Generación

**max_length:**
- Longitud **total** (incluye el prompt)
- Si el prompt tiene 5 tokens y max_length=50, generará hasta 45 tokens nuevos

**num_return_sequences:**
- Cantidad de variaciones a generar
- Útil para explorar diferentes continuaciones

**temperature:**
- Controla aleatoriedad/creatividad
- Valores bajos (0.3-0.7): Más predecible y coherente
- Valores altos (1.0-1.5): Más creativo pero potencialmente menos coherente

**top_k y top_p:**
- `top_k`: Solo considera los k tokens más probables
- `top_p`: Solo considera tokens cuya probabilidad acumulada sea ≤ p
- Ayudan a evitar generaciones absurdas

In [19]:
# Generemos múltiples variaciones del mismo prompt
prompt_creativo = "Un día descubrí que"

resultados = generator(
    prompt_creativo,
    max_length=60,
    num_return_sequences=3,  # Genera 3 variaciones
    do_sample=True,
    temperature=0.9,
    top_k=50,               # Solo considera top 50 tokens
    top_p=0.95              # Nucleus sampling
)

print(f"Prompt: '{prompt_creativo}'\n")
print("Variaciones generadas:")
print("=" * 80)

for i, resultado in enumerate(resultados, 1):
    print(f"\nVariación {i}:")
    print(resultado['generated_text'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: 'Un día descubrí que'

Variaciones generadas:

Variación 1:
Un día descubrí que había comprado una pila de caramelos de avena por error. Los caramelos de avena son una base para una bebida popular en ciertos países, por lo que pensé que ya tenía lo que busaba. En realidad, tenía una barra de chocolate salado con avena, un sabor delicioso pero que no era lo que estaba buscando.

Sin embargo, la experiencia me hizo reflexionar sobre la importancia de la atención al detalle en la compra en línea. En este caso, me equivoqué al interpretar el nombre del producto sin leer las etiquetas y el enfoque en la descripción del producto. Esto me llevó a una situación ridícula, donde pensé que tenía la solución a un problema real y me frustré por no haber podido encontrar lo que estaba buscando.

En el futuro, espero que me aseguraré de leer más detenmente los detalles de las compras en línea y no asumir nada sin verificar los datos. Aunque los caramelos de avena pueden ser una delicia, no es

In [20]:
# Ejercicio: Generación libre
mi_prompt = "Tu prompt aquí"

mi_generacion = generator(
    mi_prompt,
    max_length=100,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)

print(mi_generacion[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Tu prompt aquí!

En la historia de la pintura, hay muchos artistas que han sido reconocidos por su trabajo, pero un artista que a menudo pasa desapercibido es el pintor William Hogarth (1697-1764). Aunque es famoso por sus dibujos satíricos y sus escenas de la vida cotidiana, su obra a menudo se ve relegada al margen en comparación con los grandes maestros del siglo XVIII como Vermeer o Rembrandt.

Hogarth fue un artista inglés del siglo XVIII que se especializó en la pintura de escenas de la vida cotidiana y de temas morales. Su estilo se caracteriza por su uso de colores vivos y su atención al detalle, que le permitió capturar la esencia de la vida diaria de la clase media en Inglaterra durante el siglo XVIII. Sus obras más famosas incluyen "Las Aves de la Noche" (1732), "La Llegada del Marqués de Waterford" (1735) y "Los Casamientos" (1743-1745).

A pesar de su importancia, Hogarth a


## Comparación de Modelos: ¿Cuándo Usar Cada Uno?

### Resumen de Modelos Usados

| Tarea | Modelo | Tipo | Uso Recomendado |
|-------|--------|------|----------------|
| Análisis de Sentimiento | BETO Sentiment | Encoder (BERT) | Clasificar opiniones, reviews, feedback |
| Clasificación Zero-Shot | BERT XNLI | Encoder (BERT) | Categorización flexible sin entrenamiento |
| Sumarización | mT5 XLSum | Encoder-Decoder | Generar resúmenes concisos de noticias/artículos |
| Traducción | MarianMT | Encoder-Decoder | Traducir entre idiomas específicos |
| Generación | GPT-2 Spanish | Decoder (GPT) | Completar texto, generar contenido creativo |

### Criterios de Selección

**Análisis vs Generación:**
- **Análisis** (clasificación, NER, etc.) → Modelos encoder (BERT)
- **Generación** (texto nuevo) → Modelos decoder (GPT)
- **Transformación** (traducción, sumarización) → Encoder-decoder (T5, BART)

**Idioma:**
- Preferir modelos entrenados específicamente en español
- Modelos multilingües funcionan pero pueden ser menos precisos

**Tamaño del Modelo:**
- Modelos grandes: Más precisos pero más lentos y requieren más memoria
- Modelos pequeños: Más rápidos pero potencialmente menos precisos
- Para producción: Balance entre precisión y latencia

**Datos Disponibles:**
- Con datos etiquetados: Entrenamiento supervisado tradicional
- Sin datos etiquetados: Zero-shot o few-shot learning
- Con pocos ejemplos: Few-shot learning o fine-tuning

## Resumen de Conceptos Clave

### Pipelines de Hugging Face

1. **Abstracción de alto nivel**: Encapsulan preprocesamiento, inferencia y postprocesamiento
2. **Fáciles de usar**: Pocas líneas de código para aplicar modelos complejos
3. **Flexibles**: Podés especificar qué modelo usar para cada tarea

### Tareas de NLP Cubiertas

1. **Análisis de Sentimiento**: Clasificar texto según emoción/opinión
   - Modelo: BETO Sentiment Analysis
   - Salida: POS/NEG/NEU con score de confianza

2. **Clasificación Zero-Shot**: Categorizar sin entrenamiento previo
   - Modelo: BERT XNLI
   - Ventaja: Categorías flexibles

3. **Sumarización**: Generar resúmenes concisos
   - Modelo: mT5 XLSum
   - Tipo: Abstractiva (genera nuevas oraciones)

4. **Traducción**: Convertir texto entre idiomas
   - Modelo: MarianMT
   - Específico por par de idiomas

5. **Generación de Texto**: Crear contenido nuevo
   - Modelo: GPT-2 Spanish
   - Continúa prompts de forma coherente

### Parámetros Importantes

- `model`: Qué modelo específico usar
- `max_length`: Longitud máxima de la salida
- `min_length`: Longitud mínima (para sumarización)
- `do_sample`: Generación determinística vs estocástica
- `temperature`: Controla creatividad (0.1-2.0)
- `top_k` / `top_p`: Limitan tokens candidatos para mejor calidad
- `num_return_sequences`: Cantidad de variaciones a generar

## Guía de Preguntas y Respuestas

### Preguntas de Comprensión

**1. ¿Qué es un pipeline de Hugging Face y qué ventajas ofrece?**

<details>
<summary>Ver respuesta</summary>

Un **pipeline** es una interfaz de alto nivel que encapsula todo el flujo de trabajo necesario para aplicar un modelo de NLP:

1. **Preprocesamiento**: Tokenización y conversión a formato que el modelo entiende
2. **Inferencia**: Ejecución del modelo
3. **Postprocesamiento**: Conversión de la salida a formato legible

**Ventajas:**
- **Simplicidad**: Usar modelos complejos con pocas líneas de código
- **Abstracción**: No necesitás entender todos los detalles de implementación
- **Consistencia**: API uniforme para diferentes tareas
- **Flexibilidad**: Podés especificar qué modelo usar o dejar que elija automáticamente

Ejemplo:
```python
# Sin pipeline (complejo)
tokenizer = AutoTokenizer.from_pretrained("modelo")
model = AutoModelForSequenceClassification.from_pretrained("modelo")
inputs = tokenizer(texto, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.softmax(outputs.logits, dim=-1)

# Con pipeline (simple)
classifier = pipeline("sentiment-analysis", model="modelo")
resultado = classifier(texto)
```
</details>

---

**2. ¿Por qué es importante usar modelos entrenados específicamente en español en lugar de modelos multilingües genéricos?**

<details>
<summary>Ver respuesta</summary>

Los modelos entrenados específicamente en español ofrecen varias ventajas:

**1. Mejor comprensión de variantes regionales:**
- Modismos argentinos, mexicanos, españoles
- Vocabulario específico de cada región
- Ejemplo: "colectivo" (AR) vs "camión" (MX) vs "autobús" (ES)

**2. Manejo superior de morfología española:**
- Conjugaciones verbales complejas
- Género gramatical
- Acentuación y diacríticos (ñ, á, é)

**3. Mayor cobertura del vocabulario:**
- Palabras específicas del español no fragmentadas en subpalabras
- Tokenización más eficiente

**4. Rendimiento superior:**
- Modelos multilingües distribuyen su capacidad entre muchos idiomas
- Modelos específicos concentran toda su capacidad en un idioma

**Cuándo usar multilingües:**
- Cuando necesitás trabajar con múltiples idiomas simultáneamente
- Cuando no existe un modelo específico de calidad para tu idioma
- Para tareas de traducción o comparación entre idiomas
</details>

---

**3. ¿Qué es la clasificación zero-shot y cómo funciona internamente?**

<details>
<summary>Ver respuesta</summary>

La **clasificación zero-shot** permite categorizar textos en clases que el modelo nunca vio durante su entrenamiento.

**Funcionamiento:**

El modelo fue entrenado en **Natural Language Inference (NLI)**, que consiste en determinar si una hipótesis se deduce de una premisa:

```
Premisa: "Un perro está corriendo en el parque"
Hipótesis: "Un animal está al aire libre"
Resultado: Implicación (la hipótesis se deduce de la premisa)
```

Para clasificación:
1. Tu texto es la "premisa"
2. Cada etiqueta se convierte en "hipótesis":
   - Texto: "Messi marcó tres goles"
   - Etiqueta "deportes" → Hipótesis: "Este texto trata sobre deportes"
   - Etiqueta "política" → Hipótesis: "Este texto trata sobre política"
3. El modelo calcula scores de implicación para cada hipótesis
4. La etiqueta con mayor score es la categoría predicha

**Ventajas:**
- No requiere datos de entrenamiento etiquetados
- Categorías flexibles (podés cambiarlas sin reentrenar)
- Ideal para prototipado rápido

**Limitaciones:**
- Menos preciso que modelos entrenados específicamente
- Sensible a cómo formulás las etiquetas
- Más lento que clasificadores tradicionales
</details>

---

**4. ¿Cuál es la diferencia entre sumarización extractiva y abstractiva?**

<details>
<summary>Ver respuesta</summary>

**Sumarización Extractiva:**
- Selecciona oraciones o frases del texto original
- No genera nuevo texto
- Copia literalmente partes del documento

Ejemplo:
```
Original: "El perro corrió por el parque. El gato dormía en casa. Hacía mucho sol."
Extractiva: "El perro corrió por el parque. Hacía mucho sol."
```

**Sumarización Abstractiva:**
- Genera nuevas oraciones que capturan la esencia del original
- Puede parafrasear, combinar información, usar sinónimos
- Más similar a cómo un humano resumiría

Ejemplo:
```
Original: "El perro corrió por el parque. El gato dormía en casa. Hacía mucho sol."
Abstractiva: "Mientras el perro disfrutaba del sol en el parque, el gato descansaba en casa."
```

**Comparación:**

| Aspecto | Extractiva | Abstractiva |
|---------|-----------|-------------|
| Complejidad | Simple | Compleja |
| Modelos | No requiere LLMs | Requiere Transformers |
| Fidelidad | Alta (copia literal) | Variable (puede parafrasear incorrectamente) |
| Naturalidad | Menos natural | Más natural |
| Concisión | Limitada | Alta |

El modelo mT5 XLSum que usamos es **abstractivo**: genera resúmenes nuevos en lugar de copiar oraciones.
</details>

---

**5. ¿Qué es el parámetro temperature y cómo afecta la generación de texto?**

<details>
<summary>Ver respuesta</summary>

**Temperature** controla la aleatoriedad/creatividad en la generación de texto modificando la distribución de probabilidad de los tokens candidatos.

**Cómo funciona:**

Antes de aplicar softmax a los logits, se dividen por la temperatura:

```python
logits_ajustados = logits / temperature
probabilidades = softmax(logits_ajustados)
```

**Efecto según el valor:**

**Temperature < 1 (ej: 0.3, 0.5):**
- Acentúa diferencias: Tokens probables se vuelven MÁS probables
- Generación más conservadora y predecible
- Menos errores pero más repetitivo
- Útil para: Tareas que requieren precisión (respuestas técnicas, código)

**Temperature = 1:**
- No modifica las probabilidades originales del modelo
- Comportamiento "natural" del modelo

**Temperature > 1 (ej: 1.2, 1.5):**
- Suaviza diferencias: Tokens improbables ganan probabilidad
- Generación más aleatoria y creativa
- Más variedad pero mayor riesgo de incoherencias
- Útil para: Escritura creativa, brainstorming, múltiples alternativas

**Ejemplo visual:**
```
Logits originales: ["Buenos": 3.0, "La": 1.0, "Otra": 0.5]

Temperature 0.5 (conservador):
  Probabilidades: ["Buenos": 0.95, "La": 0.04, "Otra": 0.01]
  → Casi siempre elige "Buenos"

Temperature 1.0 (neutral):
  Probabilidades: ["Buenos": 0.82, "La": 0.13, "Otra": 0.05]

Temperature 1.5 (creativo):
  Probabilidades: ["Buenos": 0.65, "La": 0.22, "Otra": 0.13]
  → Más probable explorar alternativas
```

**Recomendaciones:**
- Tareas formales: 0.3-0.7
- Uso general: 0.7-1.0
- Creatividad: 1.0-1.3
- Experimentación: > 1.3 (cuidado con incoherencias)
</details>

---

### Preguntas de Aplicación

**6. Tenés un dataset de reviews de productos sin etiquetar. ¿Qué pipeline usarías y cómo lo implementarías?**

<details>
<summary>Ver respuesta</summary>

Para analizar reviews sin etiquetar, usaría el pipeline de **análisis de sentimiento**:

```python
from transformers import pipeline
import pandas as pd

# Cargar modelo de sentimiento
sentiment = pipeline(
    "sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis"
)

# Cargar reviews
reviews = pd.read_csv("reviews.csv")

# Función para analizar
def analizar_review(texto):
    resultado = sentiment(texto)[0]
    return resultado['label'], resultado['score']

# Aplicar a todo el dataset
reviews['sentimiento'], reviews['confianza'] = zip(
    *reviews['texto'].apply(analizar_review)
)

# Análisis agregado
print("Distribución de sentimientos:")
print(reviews['sentimiento'].value_counts())

print("\nConfianza promedio por sentimiento:")
print(reviews.groupby('sentimiento')['confianza'].mean())

# Identificar reviews ambiguos (baja confianza)
reviews_ambiguos = reviews[reviews['confianza'] < 0.7]
print(f"\nReviews ambiguos: {len(reviews_ambiguos)}")
```

**Optimizaciones:**
- Procesar en batches para mayor velocidad
- Filtrar reviews muy cortos (< 10 palabras) que pueden dar falsos positivos
- Revisar manualmente los casos de baja confianza
</details>

---

**7. Necesitás categorizar miles de noticias en 20 categorías diferentes. ¿Usarías zero-shot o entrenarías un clasificador? Justificá.**

<details>
<summary>Ver respuesta</summary>

**Respuesta corta:** Depende de si tenés datos etiquetados y requisitos de precisión.

**Usar Zero-Shot cuando:**
- No tenés datos etiquetados (o muy pocos)
- Las categorías cambian frecuentemente
- Es un prototipo o exploración inicial
- Precisión moderada es aceptable (80-85%)

```python
# Zero-shot
classifier = pipeline(
    "zero-shot-classification",
    model="Recognai/bert-base-spanish-wwm-cased-xnli"
)

categorias = ["política", "economía", "deportes", ...] # 20 categorías

resultado = classifier(noticia, candidate_labels=categorias)
```

**Entrenar clasificador cuando:**
- Tenés cientos/miles de ejemplos etiquetados por categoría
- Necesitás alta precisión (> 90%)
- Las categorías son fijas
- Vas a clasificar millones de documentos (el costo de entrenamiento se amortiza)

```python
# Clasificador entrenado
from transformers import AutoModelForSequenceClassification, Trainer

# Fine-tune BETO con tus datos
model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased",
    num_labels=20
)

trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()
```

**Enfoque híbrido (recomendado):**
1. Empezar con zero-shot para explorar y obtener primeras etiquetas
2. Revisar manualmente algunos casos por categoría
3. Usar esas etiquetas para entrenar un clasificador
4. Evaluar si la mejora justifica el esfuerzo

**Para tu caso (miles de noticias, 20 categorías):**
Probablemente **entrenar es mejor** si:
- Vas a clasificar regularmente (no es una tarea única)
- Podés etiquetar al menos 50-100 ejemplos por categoría
- La precisión es importante
</details>

---

**8. ¿Cómo implementarías un sistema que traduce automáticamente noticias de español a inglés y genera un resumen en inglés?**

<details>
<summary>Ver respuesta</summary>

Implementación de un pipeline de traducción + sumarización:

```python
from transformers import pipeline

# Opción 1: Traducir primero, luego resumir
def traducir_y_resumir_v1(texto_espanol):
    # 1. Traducir español → inglés
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-es-en"
    )
    texto_ingles = translator(texto_espanol)[0]['translation_text']
    
    # 2. Resumir en inglés
    summarizer = pipeline(
        "summarization",
        model="facebook/bart-large-cnn"  # Modelo de sumarización en inglés
    )
    resumen_ingles = summarizer(
        texto_ingles,
        max_length=100,
        min_length=30
    )[0]['summary_text']
    
    return resumen_ingles

# Opción 2: Resumir primero (en español), luego traducir
def traducir_y_resumir_v2(texto_espanol):
    # 1. Resumir en español
    summarizer_es = pipeline(
        "summarization",
        model="csebuetnlp/mT5_multilingual_XLSum"
    )
    resumen_espanol = summarizer_es(
        texto_espanol,
        max_length=100,
        min_length=30
    )[0]['summary_text']
    
    # 2. Traducir resumen
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-es-en"
    )
    resumen_ingles = translator(resumen_espanol)[0]['translation_text']
    
    return resumen_ingles

# Uso
noticia = """
Tu noticia larga en español aquí...
"""

print("Opción 1 (traducir → resumir):")
print(traducir_y_resumir_v1(noticia))

print("\nOpción 2 (resumir → traducir):")
print(traducir_y_resumir_v2(noticia))
```

**¿Cuál opción es mejor?**

**Opción 1 (traducir → resumir):**
- Ventaja: Resumen generado por modelo especializado en inglés (mejor calidad)
- Desventaja: Traducir todo el texto es más lento y costoso

**Opción 2 (resumir → traducir):**
- Ventaja: Más eficiente (resume primero, reduce tokens a traducir)
- Ventaja: Menos errores de traducción (menos texto = menos oportunidades de error)
- Desventaja: Modelo de sumarización multilingüe puede ser menos preciso

**Recomendación:** Opción 2 para la mayoría de casos (más eficiente y suficientemente preciso).
</details>

---

**9. El modelo de generación produce texto repetitivo. ¿Qué parámetros ajustarías para mejorar la calidad?**

<details>
<summary>Ver respuesta</summary>

La repetición es un problema común en generación de texto. Soluciones:

**1. Ajustar temperature:**
```python
generator(
    prompt,
    temperature=0.8,  # Aumentar de 0.5 a 0.8-1.0
    do_sample=True
)
```
Introduce más variedad en la selección de tokens.

**2. Usar repetition_penalty:**
```python
generator(
    prompt,
    repetition_penalty=1.2,  # Penaliza tokens ya usados (1.0-2.0)
    do_sample=True
)
```
Reduce la probabilidad de tokens que ya aparecieron.

**3. Activar top_k y top_p:**
```python
generator(
    prompt,
    top_k=50,      # Solo considera top 50 tokens
    top_p=0.95,    # Nucleus sampling
    do_sample=True
)
```
Evita tokens muy improbables pero permite variedad.

**4. Ajustar no_repeat_ngram_size:**
```python
generator(
    prompt,
    no_repeat_ngram_size=3  # No repite secuencias de 3 tokens
)
```
Previene repetición de frases completas.

**Configuración completa anti-repetición:**
```python
resultado = generator(
    prompt,
    max_length=100,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3
)
```

**Advertencia:** Valores muy altos de `repetition_penalty` (> 1.5) pueden generar texto incoherente.
</details>

---

**10. Implementá un sistema que analice el sentimiento de cada oración en un texto largo y determine el sentimiento predominante.**

<details>
<summary>Ver respuesta</summary>

```python
from transformers import pipeline
import spacy

# Cargar modelos
nlp = spacy.load("es_core_news_sm")  # Para segmentación de oraciones
sentiment = pipeline(
    "sentiment-analysis",
    model="finiteautomata/beto-sentiment-analysis"
)

def analizar_sentimiento_detallado(texto):
    """
    Analiza el sentimiento oración por oración.
    
    Returns:
    - sentimientos_por_oracion: Lista de (oración, sentimiento, score)
    - sentimiento_predominante: POS/NEG/NEU
    - distribucion: Conteo de cada sentimiento
    """
    # 1. Segmentar en oraciones
    doc = nlp(texto)
    oraciones = [sent.text.strip() for sent in doc.sents]
    
    # 2. Analizar cada oración
    sentimientos = []
    conteo = {'POS': 0, 'NEG': 0, 'NEU': 0}
    
    print("Análisis por oración:")
    print("=" * 80)
    
    for i, oracion in enumerate(oraciones, 1):
        if len(oracion.split()) < 3:  # Skip oraciones muy cortas
            continue
            
        resultado = sentiment(oracion)[0]
        sentimiento_label = resultado['label']
        score = resultado['score']
        
        sentimientos.append((oracion, sentimiento_label, score))
        conteo[sentimiento_label] += 1
        
        print(f"\n{i}. {oracion}")
        print(f"   Sentimiento: {sentimiento_label} ({score:.3f})")
    
    # 3. Determinar sentimiento predominante
    predominante = max(conteo, key=conteo.get)
    
    # 4. Calcular score promedio por categoría
    scores_por_categoria = {'POS': [], 'NEG': [], 'NEU': []}
    for _, label, score in sentimientos:
        scores_por_categoria[label].append(score)
    
    promedios = {
        label: sum(scores)/len(scores) if scores else 0
        for label, scores in scores_por_categoria.items()
    }
    
    # 5. Resultados
    print("\n" + "="*80)
    print("\nRESUMEN:")
    print(f"Total de oraciones analizadas: {len(sentimientos)}")
    print(f"\nDistribución:")
    for label, count in conteo.items():
        porcentaje = (count / len(sentimientos)) * 100 if sentimientos else 0
        print(f"  {label}: {count} ({porcentaje:.1f}%)")
    
    print(f"\nConfianza promedio por categoría:")
    for label, avg in promedios.items():
        if avg > 0:
            print(f"  {label}: {avg:.3f}")
    
    print(f"\nSentimiento predominante: {predominante}")
    
    return sentimientos, predominante, conteo

# Ejemplo de uso
texto_ejemplo = """
La película empezó bien, con una introducción prometedora. Los primeros veinte minutos
fueron excelentes. Sin embargo, después la trama se volvió confusa y aburrida. Los
personajes no estaban bien desarrollados. El final fue decepcionante. No la recomendaría.
"""

sentimientos, predominante, distribucion = analizar_sentimiento_detallado(texto_ejemplo)
```

**Extensiones:**
- Visualizar evolución del sentimiento a lo largo del texto
- Identificar puntos de inflexión (cambios de sentimiento)
- Ponderar por la confianza del modelo
</details>

---

### Preguntas de Reflexión

**11. ¿Qué consideraciones éticas hay al usar análisis de sentimiento automático para evaluar opiniones de usuarios?**

<details>
<summary>Ver respuesta</summary>

Consideraciones éticas importantes:

**1. Sesgos del modelo:**
- Los modelos reflejan sesgos de sus datos de entrenamiento
- Pueden malinterpretar expresiones de grupos minoritarios
- Sensibilidad cultural: Expresiones positivas en una cultura pueden ser neutrales en otra

**2. Contexto y sarcasmo:**
- Los modelos a veces fallan con sarcasmo, ironía, humor
- "Genial, otra vez se cayó el servidor" → Puede detectarse como positivo
- Riesgo: Tomar decisiones basadas en análisis incorrectos

**3. Privacidad:**
- ¿Los usuarios saben que sus textos serán analizados automáticamente?
- ¿Se anonimiza la información antes del análisis?
- ¿Se almacenan los textos originales?

**4. Consecuencias de errores:**
- Filtrar reviews legítimas negativas como "spam negativo"
- Discriminar candidatos en procesos de selección
- Decisiones comerciales basadas en análisis sesgados

**5. Transparencia:**
- Los usuarios deberían saber que están siendo analizados por IA
- Las empresas deberían explicar cómo se usa el análisis
- Debe haber opción de apelación/revisión humana

**6. Reduccionismo:**
- Reducir opiniones complejas a "positivo/negativo/neutral" pierde matices
- Las emociones humanas son multidimensionales

**Mejores prácticas:**
- Usar análisis automático como apoyo, no como decisión final
- Incluir revisión humana para casos críticos
- Auditar regularmente el modelo para detectar sesgos
- Ser transparente sobre las limitaciones
- Permitir que los usuarios corrijan clasificaciones erróneas
</details>

---

**12. Los modelos de traducción automática están reemplazando traductores humanos en algunos contextos. ¿Cuáles son las limitaciones fundamentales que hacen que los traductores humanos sigan siendo necesarios?**

<details>
<summary>Ver respuesta</summary>

Limitaciones fundamentales de la traducción automática:

**1. Contexto cultural profundo:**
- Modismos, refranes, expresiones culturales
- Ejemplo: "Estoy como piña" (AR: mal) vs "cool como piña" (no tiene sentido en inglés)
- Referencias históricas, culturales, literarias específicas

**2. Ambigüedad y polisemia:**
- "Banco" puede ser institución financiera, asiento, o banca de datos
- Los humanos usan contexto pragmático y conocimiento del mundo
- Modelos dependen solo de contexto lingüístico limitado

**3. Tono y registro:**
- Formal vs informal, técnico vs coloquial
- Traducir "You" al español: ¿tú, vos, usted?
- El tono puede cambiar completamente el significado social

**4. Creatividad y estilo:**
- Traducción literaria requiere recrear efectos artísticos
- Poesía: Ritmo, rima, métricas
- Juegos de palabras, ironía, dobles sentidos

**5. Conocimiento especializado:**
- Textos legales: Terminología precisa con implicaciones legales
- Textos médicos: Errores pueden tener consecuencias graves
- Textos técnicos: Requieren comprensión del dominio

**6. Adaptación al propósito:**
- Traducción para doblaje (debe coincidir con movimientos labiales)
- Localización de software (adaptar no solo idioma sino UX)
- Marketing: Adaptar mensajes a audiencias específicas

**7. Ética y sensibilidad:**
- Contenido sensible (traumas, violencia, temas delicados)
- Evitar lenguaje ofensivo o inapropiado en la cultura destino
- Respetar intenciones del autor original

**Casos donde IA es suficiente:**
- Traducción de documentos informativos básicos
- Comunicación informal (emails, chats)
- Traducción de noticias con contexto claro
- Primera pasada para que traductores humanos refinen

**Casos donde humanos son esenciales:**
- Literatura, poesía, textos creativos
- Documentos legales y médicos
- Marketing y contenido persuasivo
- Contenido culturalmente sensible
- Traducción simultánea (interpretación)

**Futuro probable:** Colaboración humano-IA, donde la IA genera borradores y humanos refinan, especialmente para contenido crítico o creativo.
</details>

---

### Desafíos Adicionales

**Desafío 1**: Construí un clasificador de temas de noticias usando zero-shot. Probalo con 10 noticias reales de diferentes categorías y evaluá la precisión manualmente.

**Desafío 2**: Implementá un sistema que tome reviews de productos, las analice por sentimiento, y genere un resumen automático destacando los aspectos más mencionados (positivos y negativos).

**Desafío 3**: Creá una herramienta que traduzca un texto largo del español al inglés manteniendo el formato original (párrafos, listas, etc.).

---

## Referencias y Recursos Adicionales

- [Hugging Face Model Hub](https://huggingface.co/models) - Miles de modelos preentrenados
- [Transformers Documentation](https://huggingface.co/docs/transformers/index)
- [BETO: Spanish BERT](https://github.com/dccuchile/beto)
- [MarianMT Models](https://huggingface.co/Helsinki-NLP)
- [Pipeline Task Documentation](https://huggingface.co/docs/transformers/main_classes/pipelines)
- [Zero-Shot Learning Explained](https://joeddav.github.io/blog/2020/05/29/ZSL.html)